# Исследование надежности заемщиков


## Откройте таблицу и изучите общую информацию о данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

<ipython-input-18-0bc713f3049f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['total_income_category'] = data['total_income'].apply(categorize_income)


In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [31]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
data.head() # Выведиv первые 5 строчек датафрейма data на экран.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [23]:
data.info() #Выведим основную информацию о датафрейме с помощью метода info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21331 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21331 non-null  int64  
 1   days_employed          21331 non-null  float64
 2   dob_years              21331 non-null  int64  
 3   education              21331 non-null  object 
 4   education_id           21331 non-null  int64  
 5   family_status          21331 non-null  object 
 6   family_status_id       21331 non-null  int64  
 7   gender                 21331 non-null  object 
 8   income_type            21331 non-null  object 
 9   debt                   21331 non-null  int64  
 10  total_income           21331 non-null  int64  
 11  purpose                21331 non-null  object 
 12  total_income_category  21331 non-null  object 
 13  purpose_category       21331 non-null  object 
dtypes: float64(1), int64(6), object(7)
memory usage: 2.4+ 

In [24]:
data['children'].value_counts() #отфильтруем таблицу по столбцу children с помощью метода value_counts()

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

На основании диапазонов, указанных ниже, создадим в датафрейме data столбец children_category с категориями:

0 — 'нет детей';
1 — 'один ребенок';
2 — 'два ребенка';
3 — 'многодетные';
Таким образом, мы объединим от 3 до 5 детей в обну категорию

In [25]:
def categorize_children(row):
    if row == 0:
        return 'нет детей'
    elif row == 1:
        return 'один ребенок'
    elif row == 2:
        return 'два ребенка'
    return 'многодетные'

In [26]:
data['children_category'] = data['children'].apply(categorize_children)

<ipython-input-26-50fd35a793c6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['children_category'] = data['children'].apply(categorize_children)


Создадим сводную таблицу

In [27]:
data_children = data.pivot_table(index=['children_category'], values=['debt'], aggfunc=['sum', 'count', 'mean'])
data_children.columns = ['sum', 'count', 'percent'] #переименновываю колонки, чтобы уйти от мультииндексов, иначе я не могу отсортировать таблицу по %
data_children=data_children.reset_index()
data_children['percent'] = data_children['percent'].map('{:.1%}'.format)
data_children.sort_values(by='percent')



,children_category,sum,count,percent
2,нет детей,1063,14091,7.5%
1,многодетные,31,380,8.2%
3,один ребенок,444,4808,9.2%
0,два ребенка,194,2052,9.5%


**Вывод:** **Наличие детей увеличивает % задолженности по возврату кредитов.**
Однако в многодетных семьях % должников ниже **(8,2%)**, чем в семьях с одним ребенком **(9,2%)** или с двумя детьми **(9,5%)**.
Самый низкий показатель у семей без детей **(7,5%)**.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [28]:
data_family = data.pivot_table(index=['family_status'], values=['debt'], aggfunc=['sum', 'count', 'mean'])
data_family.columns = ['sum', 'count', 'percent']
data_family=data_family.reset_index()
data_family['percent'] = data_family['percent'].map('{:.1%}'.format)
data_family.sort_values(by='percent')

,family_status,sum,count,percent
2,вдовец / вдова,63,951,6.6%
1,в разводе,84,1189,7.1%
4,женат / замужем,927,12261,7.6%
3,гражданский брак,385,4134,9.3%
0,Не женат / не замужем,273,2796,9.8%


**Вывод:** Самый низкий процент по задолжности у категории **вдовец / вдова (6,6%)**. Это может быть обусловленно получением дополнительного дохода в виде **наследства**, а так же судя по общей тенденции люди становятся вдовецами или вдовами уже в пенсионном возрасте или предпенсионном, что говорит о взрослых детях, которые могут помочь или постоянном доходе ввиде пенсии.
Самый высокий показатель у категории **Не женат / не замужем (9.8%)**. Это может быть обусловленно тем, что у них больше свободы распаряжатся своими средствами и так же в основном в эту категорю лиц входят более молодые люди, что говорит о недостаточной финансовой грамотности.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [29]:
data_income = data.pivot_table(index=['total_income_category'], values=['debt'], aggfunc=['sum', 'count', 'mean'])
data_income.columns = ['sum', 'count', 'percent']
data_income=data_income.reset_index()
data_income['percent'] = data_income['percent'].map('{:.1%}'.format)
data_income.sort_values(by='percent')

,total_income_category,sum,count,percent
3,D,21,349,6.0%
1,B,354,5014,7.1%
0,A,2,25,8.0%
2,C,1353,15921,8.5%
4,E,2,22,9.1%


**Вывод:** Самый высокий % по задолжности у категории "Е", но основываясь на таблицу выше, я не расматривал бы категорию "Е" и "А" из-за маленькой выборки, а выводы делаем опираясь на категории "C", "B", "D". Основываясь на это:
Самый высокий % по задолжности у **категории "С" (8,5%)**,
Самый низкий % по задолжности у **категории "D" (6,0%)**

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [30]:
data_purpose = data.pivot_table(index=['purpose_category'], values=['debt'], aggfunc=['sum', 'count', 'mean'])
data_purpose.columns = ['sum', 'count', 'percent']
data_purpose=data_purpose.reset_index()
data_purpose['percent'] = data_purpose['percent'].map('{:.1%}'.format)
data_purpose.sort_values(by='percent')

,purpose_category,sum,count,percent
1,операции с недвижимостью,780,10751,7.3%
3,проведение свадьбы,183,2313,7.9%
0,операции с автомобилем,400,4279,9.3%
2,получение образования,369,3988,9.3%


**Вывод:** Операясь на дынные можно сделать вывод, что самый большой % задолжности у лиц бравшие кредиты на **получение образования (9.3%)** и **операции с автомобилем	(9.3%)**

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* Неверный ввод данных, повреждение файла данных при сохранении, технические проблемы.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Медианное значение лучше всего отражает выборку в целом, в случае, если есть незаполненные данные. Среднее арифметическое отражает ситуацию некорректно, так как не учитывает разброс данных.

### Шаг 4: общий вывод.

1.Наличие детей увеличивает % задолженности по возврату кредитов. Самый большой % у холостых(Не женат / не замужем) и лиц живущих в Гражданском браке. Лица с доходом 50001–200000 — 'C'. Лица берущие кредиты на получение образования и операции с автомобилем чаще всего попадают в задолжность.
2.Появления пропусков в данных могут быть: неверный ввод данных, повреждение файла данных при сохранении, технические проблемы.
Заполнение пропусков медианой является лучшим решением из-за устойчивости медианы к выбросам и аномальным значениям.
